# 1. Introduction

$ \text{For this project, we are going to price the following contract who will pay at maturity T in USD.}$
<br><br>
$$ max\left[0,\left(\frac{S(T)}{S(0)}-k\right)\left(\frac{L(T-\delta,T-\delta,T)}{L(0,T-\delta,T)}-k^\prime\right)\right]$$
<br>
$\text{where:} $<br>
$\text{S(t) = the stock price of STOXX50E quoted from EUR into USD}$ <br>
$\text{$\delta$ = 3-month USD LIBOR rate}$ <br>
$\text{T = expiration date}$ <br>
$\text{k, $k^{'}$ = relative strike prices}$ <br>
$\text{L(t,T-$\delta$,T) = forward rate bet}$

# 2. Model Assumption

##   a) Foreign Equity Model

Assume the foreign stock follows the local volatility model with a dynamic under the foreign risk-neutral measure $Q^f$:
<br><br>
$$
dS_t = r_t^fS_t dt + \sigma(t,S_t)S_t dW_t^{1,f}
$$
<br>
where the foreign rate $r_t^f$ is constant, and we will use the SSVI volatility surface model to fit the market volatility $\sigma\left(t,S_t\right)$, which will be discussed in Part III Model Caliberation.

##   b) Rates Model 

Assume the rate follows the Hull-White model with costant coefficient $\alpha$ and time-dependant $\theta$ :
<br><br>
$$
dr_t = [\theta(t)-\alpha r_t ]dt+\sigma dW_t^2
$$
<br> where $\sigma$ is constant, and we will discuss the fitting of $\theta\left(t\right)$ and Hull-White volatility $\sigma$ later in Part III Model Caliberation.

## c) Local Volatility Quanto Process

Here, we will convert the foreign model to the local volatility model with a dynamic under the domestic risk neutral measure Q:<br>
$$
dS_t = (r_t^f -\rho_x \sigma(t,S_t)\sigma_x)S_t dt + \sigma(t,S_t)S_t dW_t^1
$$
<br> where with a constant correlation between the foreign stock and foreign exchange rate $\rho$ <br><br>
$$
dW_t^1 dW_t^2 = \rho dt
$$
<br>

# 3. Model Caliberation

## a) Data Processing

@Weimin_Wang

## c) Yield Curve 

Given available zero rates $r(0,T_i)$ for various tenors i, we have
<br><br>
$$B(0,T_i) = e^{-T_ir(0,T_i)} = e^{-y_i}$$
<br>
We transform $r(0,T_i)$ into $y_i$ and interpolate $y_i$ with regard to $T_i$ using monotomic culic spline interpolation. After interpolated $y_i$ is available for $\forall T_i$, we can derive various rates for $\forall T_i$ using the following formulas:
$$
\begin{array}{11}
    \\
     P_{\text{zero-bond}} \space & B(0,T_i) = e^{-T_ir(0,T_i)} = e^{-y_i}\\
     \text{Zero rate}& r(0,T_i) = \frac{y_i}{T_i}\\
     \text{Countinous-compounding forward rate} & F(0,T_1,T_2) = \frac{1}{T_2-T_1}\log\left(\frac{B(0,T_1)}{B(0,T_2)}\right)\\
     \text{Simple forward rate}& L(0,T_1,T_2) = \frac{1}{T_2-T_1}\frac{B(0,T_1)-B(0,T_2)}{B(0,T_2)}\\
     \text{Instantaneous forward rate}& f(0,T)= lim_{(T' - T)\to 0} F(0,T,T') = \frac{\partial y(T)}{\partial T} \\
     \text{Instantaneous spot rate}& r_0= f(0,0) = \frac{\partial y_i}{\partial T_i}\rvert_{i=0} \\
\end{array}
$$

## d) Hull White Volatility $\sigma$

The purpose of this model is to compute Hull-White volatility $\sigma$ from caplet volatility $vol$. Given caplet volatility $vol$, we can get the caplet price,
<br><br>
$$
Caplet(0,T_1,T_2,K,vol) = B(0,T_2) BS^C(K,T_1,S,vol,r)
$$
<br>
After $Caplet(0,T_1,T_2,K,vol)$ is known, we can compute the bond price $C^B(0,T_1,T_2,K^B)$ and then derive the bond volatility $vol^B$ using the following equation:
<br><br>
$$
C^B(0,T_1,T_2,K^B) = \frac{(T_2-T_1)}{B(0,T_1)(1+K(T_2-T_1))}Caplet(0,T_1,T_2,K,vol) = BS^P(K^B,T_1,S^B,vol^B,r)
$$
<br>
Then we will have
<br><br>
$$
\sigma_p =\sqrt {T_1} vol^B
$$
<br>which will generate the Hull-White volatility:
<br><br>
$$
\sigma = \sqrt{\frac{2a^3\sigma_p^2}{(1-e^{-a(T_2-T_1)})^2(1-e^{-2aT_1})^2}}
$$
where <br>
$S=L(0,T_1,T_2)$ <br>
$K^B=\frac{1}{1+K(T_2-T_1)}$ <br>
$S^B=\frac{1}{1+(T_2-T_1)L(0,T_1,T_2)}$ <br>
$r = 0$ <br>
$a = 3$

## e) SSVI Volatility Surface

### * SSVI Parameterization

We consider the following SSVI parameterisation of the total implied variance surface:
<br><br>
$$
w(k,\theta_t)
= \frac{\theta_t}{2}\left\{1+\rho\varphi(\theta_t) k + \sqrt{\left(\varphi(\theta_t){k}+\rho\right)^2
+1-\rho^2 }\right\},
$$
<br>
with $\theta_t = \sigma_s^2 t$ and $k=log\frac{K}{F}$, where $\sigma_s$ denotes at-the-money volatility skew, K denotes strike, and F denotes the forward.

<br><br>
For Heston-Like Surface, the function $\varphi$ is defined as:
<br><br>
$$
\varphi(\theta)=\frac{1}{\lambda\theta}\left\{1-\frac{1-\mathrm{e}^{-\lambda\theta}}{\lambda\theta}\right\},
$$
<br>
where $\lambda>0$, $\rho \in (-1,1)$.
We further need to impose $\lambda \geq \frac{1}{4}\left(1+|\rho|\right)$ in order to prevent arbitrage.

<br><br>
and for Power-Law Surface:
<br>
$$
\varphi(\theta)=\frac{\eta}{\theta^\gamma\left(1+\theta\right)^{1-\gamma}},
$$
<br>
where $\gamma\in (0,\frac{1}{2}]$,  $\rho \in (-1,1)$.
We further need to impose $\eta\left(1+|\rho|\right)\leq 2$ in order to prevent arbitrage.



### *Opitimize the Parameters

In order to fit the whole surface, we first need to find the optimal parameters $\rho$,$\gamma$,$\lambda$,$\eta$, which can be solved by minimizing the loss function as following.
<br><br>
The basic formula for the loss function is
$$
min \sum_{t_i}\sum_{i=1}^n\left(w_{t_i,i}^{SSVI}-w_{t_i,i}^{market}\right)^2
$$
<br><br>
After adjusting the coefficients, we get
$$
min \sum_{t_i}\sum_{i=1}^n\frac{1}{2}\left(\frac{w_{t_i,i}^{SSVI}}{t_i}-\left(\sigma_{t_i,i}^{market}\right)^2\right)^2
$$
<br>
or
<br>
$$
min \sum_{t_i}\sum_{i=1}^n\frac{1}{2}\left(w_{t_i,i}^{SSVI}-\left(\sigma_{t_i,i}^{market}\right)^2t_i\right)^2
$$

## *Derivatives

Compute the first and second derivative of the SSVI function, $w(k,\theta_t)$, with respect to $k$.
<br><br>
\begin{align}
\frac{\partial w}{\partial k} & = \frac{\theta\varphi}{2}\frac{k\varphi+\rho+\rho\sqrt{\varphi^2k^2+2\rho\varphi+1}}{\sqrt{\varphi^2k^2+2k\rho\varphi+1}}\\
\frac{\partial^2 w}{\partial k^2} & = \frac{\theta\varphi^2}{2}\frac{1-\rho^2}
{\left(\varphi^2k^2+2k\rho\varphi+1\right)^\frac{3}{2}}
\end{align}
<br><br>
Use the chain rule to find the first derivative of the SSVI function, $w(k,\theta_t)$, with respect to $t$
<br><br>
$$
\frac{\partial w}{\partial t}=\frac{\partial w}{\partial \theta}\frac{\partial \theta}{\partial t}+\frac{\partial w}{\partial \varphi}\frac{\partial \varphi}{\partial \theta}\frac{\partial \theta}{\partial t}
$$
<br>
where $\frac{\partial \theta}{\partial t}$ can be calculated by interpolation and
<br>
$$
\begin{align}
\frac{\partial w}{\partial \theta} & =\frac{w}{\theta}\\
\frac{\partial w}{\partial \varphi} & = \frac{k\theta}{2}\frac{k\varphi+\rho+\rho\sqrt{\varphi^2k^2+2\rho\varphi+1}}{\sqrt{\varphi^2k^2+2k\rho\varphi+1}}\\
\frac{\partial \varphi^{power}}{\partial \theta} & =  \frac{\eta\left(\theta+\gamma\right)\left(1+\theta\right)^{\gamma-2}}{\theta^{1+\gamma}}\\
\frac{\partial \varphi^{heston}}{\partial \theta} & = \frac{2-2\exp{-\lambda\theta}-\lambda\theta\exp{-\lambda\theta}-\lambda\theta}{\lambda^2\theta^3}
\end{align}
$$




## * Local Variance and Implied Variance

<br><br>
We then compute the (Dupire) local variance via the following formula:
<br><br>
$$
\sigma_{\mathrm{loc}}^2(x, T) = \frac{\partial_{t}w(k, \theta_{t})}{g(k, w(k, \theta_{t}))},
$$
<br>
for all $x\in\mathbb{R}$ and $t\geq 0$, where the function $g$ is defined as:
<br><br>
$$
\begin{equation}\label{gFunction}
\left.
g(x,T):=\left(\left(1-\frac{x\partial_x w}{w}\right)^2 -\frac{(\partial_x w)^2}{4}\left(\frac{1}{4}+\frac{1}{w}\right) + \frac{\partial_{xx}w}{2}\right)
\right|_{(x,T)}.
\end{equation}
$$
<br>
and the implied variance is
<br><br>
$$
\sigma_{\mathrm{implied}}^2(x, T) = \frac{w(k, \theta_{t})}{T},
$$

# 4. Implementation

# 5. Stochastic Control

# 6. Conclusion